# Azure Cognitive Services

This toolkit is used to interact with the `Azure Cognitive Services API` to achieve some multimodal capabilities.

Currently There are four tools bundled in this toolkit:
- AzureCogsImageAnalysisTool: used to extract caption, objects, tags, and text from images. (Note: this tool is not available on Mac OS yet, due to the dependency on `azure-ai-vision` package, which is only supported on Windows and Linux currently.)
- AzureCogsFormRecognizerTool: used to extract text, tables, and key-value pairs from documents.
- AzureCogsSpeech2TextTool: used to transcribe speech to text.
- AzureCogsText2SpeechTool: used to synthesize text to speech.

First, you need to set up an Azure account and create a Cognitive Services resource. You can follow the instructions [here](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cwindows) to create a resource. 

Then, you need to get the endpoint, key and region of your resource, and set them as environment variables. You can find them in the "Keys and Endpoint" page of your resource.

In [ ]:
# !pip install --upgrade azure-ai-formrecognizer > /dev/null
# !pip install --upgrade azure-cognitiveservices-speech > /dev/null

# For Windows/Linux
# !pip install --upgrade azure-ai-vision > /dev/null

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["AZURE_COGS_KEY"] = ""
os.environ["AZURE_COGS_ENDPOINT"] = ""
os.environ["AZURE_COGS_REGION"] = ""

## Create the Toolkit

In [2]:
from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit

toolkit = AzureCognitiveServicesToolkit()

In [3]:
[tool.name for tool in toolkit.get_tools()]

['azure_cognitive_services_form_recognizer',
 'azure_cognitive_services_speech2text',
 'azure_cognitive_services_text2speech',
 'azure_cognitive_services_image_analysis']

## Use within an Agent

In [4]:
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType

In [5]:
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [6]:
agent.run(
    "What can I make with these ingredients?"
    "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
)



> Entering new AgentExecutor chain...

Action:
```
{
  "action": "azure_cognitive_services_image_analysis",
  "action_input": "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
}
```

Observation: Caption: a group of eggs and flour in bowls
Objects: Egg, Egg, Food
Tags: dairy, ingredient, indoor, thickening agent, food, mixing bowl, powder, flour, egg, bowl
Thought: I can use the objects and tags to suggest recipes
Action:
```
{
  "action": "Final Answer",
  "action_input": "Based on the objects and tags in the image, you could make a cake, pancakes, omelette, or quiche."
}
```

> Finished chain.


'Based on the objects and tags in the image, you could make a cake, pancakes, omelette, or quiche.'

In [7]:
audio_file = agent.run("Tell me a joke and read it out for me.")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "azure_cognitive_services_text2speech",
  "action_input": "Why did the chicken cross the road? To get to the other side!"
}
```




Observation: Speech has been generated
Thought: I need to read out the joke
Action:
```
{
  "action": "Final Answer",
  "action_input": "Why did the chicken cross the road? To get to the other side!"
}
```


> Finished chain.


In [8]:
tts = toolkit.get_tools()[-2]
path = tts.generate_and_save("Hello world")
tts.load_and_play(path)